<a href="https://colab.research.google.com/github/Amplil/nlp-test/blob/master/transformer_kyoto_ja_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

transformerによる日本語から英語への機械翻訳

コーパスは[http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz](http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz)を使う。

In [ ]:
ls

drive/  sample_data/


In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")
import slacknotice # オリジナルモジュール

## 毎回必要なインストール

In [ ]:
pip install OpenNMT-py

     |████████████████████████████████| 212 kB 9.2 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 636 kB 13.9 MB/s 
     |████████████████████████████████| 14.3 MB 155 kB/s 
     |████████████████████████████████| 73 kB 2.2 MB/s 
     |████████████████████████████████| 58 kB 7.0 MB/s 
     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
     |████████████████████████████████| 1.2 MB 30.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
  Attem

## 前処理

In [ ]:
!onmt_build_vocab -config data/transformer_kyoto_en_ja.yml

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-07-21 14:37:55,824 INFO] Counter vocab from 5000 samples.
[2021-07-21 14:37:55,824 INFO] Build vocab on 5000 transformed examples/corpus.
[2021-07-21 14:37:55,834 INFO] corpus_1's transforms: TransformPipe()
[2021-07-21 14:37:55,838 INFO] Loading ParallelCorpus(data/kftt-data-1.0/data/tok/kyoto-train.en, data/kftt-data-1.0/data/tok/kyoto-train.ja, align=None)...
[2021-07-21 14:37:57,340 INFO] Counters src:13637
[2021-07-21 14:37:57,340 INFO] Counters tgt:12199


In [ ]:
# 恐らく前のバージョンでのコマンドであるため実行できない
#!python OpenNMT-py/preprocess.py -train_src '/content/OpenNMT-py/data/'src-train.txt -train_tgt '/content/OpenNMT-py/data/'tgt-train.txt -valid_src '/content/OpenNMT-py/data/'src-val.txt -valid_tgt '/content/OpenNMT-py/data/'tgt-val.txt -save_data '/content/OpenNMT-py/data/'demo

python3: can't open file 'OpenNMT-py/preprocess.py': [Errno 2] No such file or directory


In [ ]:
# 恐らく前のバージョンでのコマンドであるため実行できない
#!python preprocess.py -train_src train.en.atok -train_tgt train.ja.atok -valid_src dev.en.atok -valid_tgt dev.ja.atok -save_data preprocessed_dataset

python3: can't open file 'preprocess.py': [Errno 2] No such file or directory


## 学習

In [ ]:
!onmt_train -config data/transformer_kyoto_en_ja.yml

[2021-07-22 04:33:20,358 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-07-22 04:33:20,359 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-07-22 04:33:20,360 INFO] Missing transforms field for valid data, set to default: [].
[2021-07-22 04:33:20,360 INFO] Parsed 2 corpora from -data.
[2021-07-22 04:33:20,633 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-07-22 04:33:20,633 INFO] Loading vocab from text file...
[2021-07-22 04:33:20,633 INFO] Loading src vocabulary from data/kftt-data-1.0/run/example.vocab.src
[2021-07-22 04:33:20,915 INFO] Loaded src vocab has 13637 tokens.
[2021-07-22 04:33:20,921 INFO] Loading tgt vocabulary from data/kftt-data-1.0/run/example.vocab.tgt
[2021-07-22 04:33:21,207 INFO] Loaded tgt vocab has 12199 tokens.
[2021-07-22 04:33:21,212 INFO] Building fields with vocab in counters...
[2021-07-22 04:33:21,227 INFO]  * tgt vocab size: 12203.
[2021-07-22 04:33:21,243 IN

In [ ]:
slacknotice.send("train 実行完了")

slack message: train 実行完了


In [ ]:
# 恐らく前のバージョンでのコマンドであるため実行できない
# ->このコマンドをyamlファイルに置き換えたらできるかもしれない->できた
"""
!export CUDA_VISIBLE_DEVICES=0 && \
python train.py -data preprocessed_dataset -save_model save_model_name \
-layers 6 -rnn_size 512 -word_vec_size 512 \
-transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding \
-train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens \
-normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam \
-warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot \
-label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 40000 -world_size 1 -gpu_ranks 0 
"""

usage: train.py [-h] [-config CONFIG] [-save_config SAVE_CONFIG] -data DATA
                [-skip_empty_level {silent,warning,error}]
                [-transforms {bart,filtertoolong,prefix,switchout,tokendrop,tokenmask,sentencepiece,bpe,onmt_tokenize} [{bart,filtertoolong,prefix,switchout,tokendrop,tokenmask,sentencepiece,bpe,onmt_tokenize} ...]]
                [-save_data SAVE_DATA] [-overwrite] [-n_sample N_SAMPLE]
                [-dump_fields] [-dump_transforms] -src_vocab SRC_VOCAB
                [-tgt_vocab TGT_VOCAB] [-share_vocab]
                [-src_vocab_size SRC_VOCAB_SIZE]
                [-tgt_vocab_size TGT_VOCAB_SIZE]
                [-vocab_size_multiple VOCAB_SIZE_MULTIPLE]
                [-src_words_min_frequency SRC_WORDS_MIN_FREQUENCY]
                [-tgt_words_min_frequency TGT_WORDS_MIN_FREQUENCY]
                [--src_seq_length_trunc SRC_SEQ_LENGTH_TRUNC]
                [--tgt_seq_length_trunc TGT_SEQ_LENGTH_TRUNC]
                [-both_embeddings BO

## 翻訳（検証）

In [ ]:
!onmt_translate -model data/kftt-data-1.0/run/model_step_10000.pt -src data/kftt-data-1.0/data/tok/kyoto-test.en -output data/kftt-data-1.0/out.txt -gpu 0 -verbose

ストリーミング出力は最後の 5000 行に切り捨てられました。
SENT 161: ['Japanese', 'National', 'Railways', 'Steam', 'Locomotive', 'Class', 'D50']
PRED 161: 日本 <unk>
PRED SCORE: -4.1551

[2021-07-22 07:06:00,786 INFO] 
SENT 162: ['Manufactured', 'in', '1925', 'by', 'Hitachi', ',', 'Ltd', '.']
PRED 162: 1925 年 （ 大正 14 年 ） 常陸 <unk> 。
PRED SCORE: -4.5727

[2021-07-22 07:06:00,786 INFO] 
SENT 163: ['Transferred', 'from', 'the', 'Noogata', 'Engine', 'Depot', '.']
PRED 163: <unk> <unk> から 。
PRED SCORE: -5.2855

[2021-07-22 07:06:00,787 INFO] 
SENT 164: ['Registration', 'deleted', 'in', '1979', '.']
PRED 164: <unk> 年
PRED SCORE: -4.2151

[2021-07-22 07:06:00,787 INFO] 
SENT 165: ['Japanese', 'National', 'Railways', 'Steam', 'Locomotive', 'Class', 'D51']
PRED 165: 日本 <unk>
PRED SCORE: -4.1551

[2021-07-22 07:06:00,787 INFO] 
SENT 166: ['Manufactured', 'in', '1936', 'by', 'Kawasaki', 'Rolling-Stock', 'Mfg', '.', 'Co', '.']
PRED 166: 昭和 11 年 （ 1936 年 ） 、 川崎 <unk> <unk> 。
PRED SCORE: -7.8548

[2021-07-22 07:06:00,787 INFO] 
